# Ensemble learning
Tutorial pokrywa najbardziej popularne metody uczenia zespołowego w oparciu o problem klasyfikacji.

Prezentacja dostępna pod linkiem: https://docs.google.com/presentation/d/102wXkjtPx06q1AxpQK76ytD0gamjus-leKIWr7xzjDk/edit?usp=sharing  
Dokumentacja używanych metod: https://scikit-learn.org/stable/modules/ensemble.html

In [ ]:
import random
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import pandas as pd
import seaborn as sns

from sklearn.datasets import fetch_openml
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.ensemble import BaggingClassifier, AdaBoostClassifier, RandomForestClassifier
from sklearn.metrics import f1_score

### Używając PCA transformujemy podzbiór MNIST do przestrzeni 30D

In [ ]:
mnist = fetch_openml("mnist_784", data_home="data/mnist_784", cache=True)

In [ ]:
zipped_mnist = list(zip(mnist.data, mnist.target))
mnist_random = random.sample(zipped_mnist, 10000)
x, y = zip(*(mnist_random))
x = np.asarray(x)
y = np.asarray(y)

In [ ]:
scaler = StandardScaler()
x_scaled = scaler.fit_transform(x)

pca = PCA(30)
x_pca = pca.fit_transform(x_scaled)

### Wybieramy ze zbioru dwie trudno rozdzielalne klasy. Będą to zbiory danych do następnych ćwiczeń.

In [ ]:
x_embedded = TSNE(n_components=2).fit_transform(x_pca)

In [ ]:
plt.figure(figsize=(15,10))

for c, ids in zip(mcolors.TABLEAU_COLORS, [str(i) for i in range(10)]):
    plt.scatter(x_embedded[y == ids, 0], x_embedded[y == ids, 1], c=c, label=ids)
plt.legend()
plt.show()

Jak widać na powyższym wykresie istnieją klasy, które wzajemnie na siebie nachodzą.  
Przykładem mogą być:  
    - 3 i 8  
    - 3 i 5  
    - 5 i 8  
    - 4 i 9  
    - 7 i 9  
Do dalszych zadań wybieramy klasę 3 i 8 

In [ ]:
digit_filter = np.where((y == '3') | (y == '8'))
x_filtered, y_filtered = x_pca[digit_filter], y[digit_filter]

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x_filtered, y_filtered, train_size=0.8)
x_train.shape

## Bagging

Każdy z 5 klasyfikatorów jest trenowany za pomocą pozdbioru o wielkości 40% calego zbioru.

https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.BaggingClassifier.html

In [ ]:
classifier = BaggingClassifier(base_estimator=SVC(), n_estimators=5, max_samples=0.4, bootstrap=True) 
classifier.fit(x_train, y_train)
classifier.score(x_test, y_test)

Definujemy fukncje pomocnicze do obliczania wyników klasyfikacji oraz ich graficznej prezentacji

In [ ]:
def ensemble(Clf, estimators=[5], samples=[0.9], features=[0.9], params={}):
    result = pd.DataFrame(columns=['n_estimators', 'max_features', 'max_samples', 'score_f1', 'score'])
    for n_estimators in estimators:
        for max_samples in samples:
            for max_features in features:
                clf = Clf(max_features=max_features, n_estimators=n_estimators, max_samples=max_samples, **params)
                clf.fit(x_train, y_train)
                pred = clf.predict(x_test)
                row = { 'n_estimators':n_estimators,
                        'max_samples':max_samples,
                        'max_features':max_features,
                        'score': clf.score(x_test, y_test),
                        'score_f1': f1_score(y_test, pred, average='weighted')
                      }
                result = result.append(row, ignore_index=True)
    return result

    
def draw_heatmap(param1, param2, df, score='score'):
    score = df.pivot(param1, param2, score)
    plt.figure(figsize=(10,10))
    sns.heatmap(score,annot=True, cmap='coolwarm')

Obserwujemy zmianę dokładności klasyfikacji i F1 dla wzrastającej liczby klasyfikatorów w zespole i liczby przykładów, na których uczone są klasyfikatory proste.

In [ ]:
estimators = range(5, 51, 5)
samples = list(map(lambda x: round(x, 1), np.arange(0.1, 1.1, 0.1)))

params = {"base_estimator": SVC(), "bootstrap": True} # bootstrap = True zapewnia losowość podzbiorów danych
result = ensemble(BaggingClassifier, estimators=estimators, samples=samples, params=params)
result

In [ ]:
# dokładność klasyfikacji
draw_heatmap('n_estimators', 'max_samples', result)

# F1
draw_heatmap('n_estimators', 'max_samples', result, score='score_f1')

### ZADANIE:

Jak zmienia się dokładność klasyfikacji dla różnych metod określania finalnej decyzji. Porównać wynik klasycznego baggingu z wynikami dla VotingClassifier(z opcją voting='hard' oraz voting='soft')

## Random Subspace
Do trenowania każdego z klasyfikatorów wykorzystany jest cały zbiór danych, lecz tylko połowa cech.

https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.BaggingClassifier.html

In [ ]:
classifier = BaggingClassifier(base_estimator=SVC(), n_estimators=5, bootstrap_features=True, max_features=0.5)
classifier.fit(x_train, y_train)
classifier.score(x_test, y_test)

Teraz obserwujemy zmienę dokładność klasyfikacji i F1 dla Random Subspace.

In [ ]:
estimators = range(5, 51, 5)
features = list(map(lambda x: round(x, 1), np.arange(0.1, 1.1, 0.1)))

params = {"base_estimator": SVC(), "bootstrap_features": True} # zapewnia losowość pozdbioru cech

result = ensemble(BaggingClassifier, estimators=estimators, features=features, params=params)

draw_heatmap('n_estimators', 'max_features', result)

draw_heatmap('n_estimators', 'max_features', result, score='score_f1')

### ZADANIE:

Jak zmienia się dokładność klasyfikacji dla różnych metod określania finalnej decyzji. Porównać wynik otrzymany dla random subspace z wynikami dla VotingClassifier(z opcją voting='hard' oraz voting='soft')

## Random Patches
Jest to połączenie klasycznego Baggingu i RandomSubspace.

Każdy z klasyfikatorów jest trenowany na podzbiorze danych oraz podzbiorze cech.

https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.BaggingClassifier.html


In [ ]:
classifier = BaggingClassifier(base_estimator=SVC(), n_estimators=5, 
                               bootstrap_features=True, max_features=0.5, max_samples=0.4)
classifier.fit(x_train, y_train)
classifier.score(x_test, y_test)

Teraz obserwujemy zmianę dokładności klasyfikacji i F1 w zależności od wielkości podzbioru danych i cech.

In [ ]:
samples = list(map(lambda x: round(x, 1), np.arange(0.1, 1.1, 0.1)))
features = list(map(lambda x: round(x, 1), np.arange(0.1, 1.1, 0.1)))

params = {"base_estimator": SVC(), "bootstrap_features": True, "bootstrap": True }

result = ensemble(BaggingClassifier, features=features, samples=samples, params=params)

draw_heatmap('max_samples', 'max_features', result)

draw_heatmap('max_samples', 'max_features', result, score='score_f1')

### ZADANIE:

Jak zmienia się dokładność klasyfikacji dla różnych metod określania finalnej decyzji. Porównać wynik otrzymany dla random patches z wynikami dla VotingClassifier(z opcją voting='hard' oraz voting='soft')

## AdaBoost
Jest to metoda, która dla nowo tworzonych kopi klasyfikatorów uwzględnia dodatkowo wagi nieprawidłowo sklasyfikowanych instancji, tak aby mogły się one skoncentrować na bardziej skomplikowanych przypadkach.  
Domyślnie jako klasyfikator bazowy używany jest DecisionTreeClassifier.

https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.AdaBoostClassifier.html

In [ ]:
classifier = AdaBoostClassifier(n_estimators=5, random_state=7)
classifier.fit(x_train, y_train)
classifier.score(x_test, y_test)

Teraz obserwujemy zmianę dokładności klasyfikacji i F1 dla AdaBoost.

In [ ]:
estimators = range(5, 101, 5)

result = pd.DataFrame(columns=['n_estimators', 'score', 'score_f1'])
for n_estimators in estimators:
    clf = AdaBoostClassifier(n_estimators=n_estimators, random_state=7)
    clf.fit(x_train, y_train)
    pred = clf.predict(x_test)
    row = { 'n_estimators':n_estimators,
            'score': clf.score(x_test, y_test),
            'score_f1': f1_score(y_test, pred, average='weighted')
          }
    result = result.append(row, ignore_index=True)
result

In [ ]:
result.score.max(), result.score.idxmax()

In [ ]:
result.score_f1.max(), result.score_f1.idxmax()

## Random Forest
Jest to metoda oparta o las drzew uczących. Każde z nich uczone jest na losowo wybranym podzbiorze danych wraz z podzbiorem cech. Skutkuje to zmniejszoną korelacją pomiędzy poszczególnymi drzewami.  
Poniżej przedstawiono wynik klasyfikacji dla 5 klasyfikatorów oraz zbiorze cech równym połowie początkowego zbioru.

https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html

In [ ]:
classifier = RandomForestClassifier(n_estimators=5, max_features=0.5)
classifier.fit(x_train, y_train)
classifier.score(x_test, y_test)

Teraz obserwujemy zmianę dokładności klasyfikacji i F1 dla Random Forest.


In [ ]:
estimators = range(5, 51, 5)
features = list(map(lambda x: round(x, 1), np.arange(0.1, 1.1, 0.1)))

result = ensemble(RandomForestClassifier, estimators=estimators, features=features)

draw_heatmap('n_estimators', 'max_features', result)

draw_heatmap('n_estimators', 'max_features', result, score='score_f1')

## Zadanie indywidualne

1. Przetransformować zbiór FMNIST używając PCA do przestrzeni 30D. Następnie wybrać 2 trudno rozdzielane klasy

2. Sprawdzić jak działa klasyczny Bagging dla 5 klasyfikatorów prostych. Następnie określić:  
    a) jak zmienia się dokładność klasyfikacji oraz F1 score dla wzrastającej liczby klasyfikatorów i wielkości podzbioru danych, na których są uczone klasyfikatory  
    b) jak zmienia się dokładność klasyfikacji dla różnych metod określania finalnej decyzji.

3. Sprawdzić jak działa RandomSubspace dla 5 klasyfikatorów prostych. Następnie określić:  
    a) jak zmienia się dokładność klasyfikacji oraz F1 score dla wzrastającej liczby klasyfikatorów i wielkości podzbioru cech, na których są uczone klasyfikatory  
    b) jak zmienia się dokładność klasyfikacji dla różnych metod określania finalnej decyzji.

4. Sprawdzić jak działa RandomPatches dla 5 klasyfikatorów prostych. Następnie określić:  
    a) jak zmienia się dokładność klasyfikacji oraz F1 score dla wzrastającej wielkości pozdzbioru danych i wielkości podzbioru cech, na których są uczone klasyfikatory  
    b) jak zmienia się dokładność klasyfikacji dla różnych metod określania finalnej decyzji.

5. Sprawdzić jak działa AdaBoost dla 5 klasyfikatorów prostych. Następnie określić:  
    a) jak zmienia się dokładność klasyfikacji oraz F1 score dla wzrastającej liczby klasyfikatorów

6. Sprawdzić jak działa RandomForest dla 5 klasyfikatorów prostych. Następnie określić:  
    a) jak zmienia się dokładność klasyfikacji oraz F1 score dla wzrastającej liczby klasyfikatorów i wielkości podzbioru cech, na których są uczone klasyfikatory